In [1]:
import pandas as pd
import shapely
from shapely.geometry import shape,mapping, Point, Polygon, MultiPolygon
shapely.speedups.enable()
import geopandas
from geopandas import GeoDataFrame
import os
import re

In [2]:
solar = pd.read_csv('pred.csv')

In [3]:
solar_median = solar.groupby(['LAT','LON'])[['pred_solar']].median().reset_index() #KWh per square meter per day

In [4]:
points = []
for lat, lon in zip(solar_median['LAT'],solar_median['LON']):
    p = Point(lon,lat)
    points.append(p)

In [5]:
poly = geopandas.GeoDataFrame.from_file('data/nga_adm_osgof_20190417_shp/nga_admbnda_adm2_osgof_20190417.shp')
poly = poly[['ADM1_EN','ADM2_EN','geometry']]

In [6]:
location = []
for point in points:
    for adm1,po in zip(poly['ADM1_EN'],poly['geometry']):
        if po.contains(point)==True:
            location.append(adm1)

In [7]:
location = pd.DataFrame(location)
location.columns = ['adm1']

In [8]:
solar_median = solar_median.merge(location, left_index=True, right_index=True)

,LAT,LON,pred_solar,adm1
0,4.85811,6.92091,2.465960,Rivers
1,4.86781,5.89871,3.367255,Bayelsa
2,4.93001,7.87221,3.024034,Akwa Ibom
3,5.43091,7.52471,1.670673,Abia
4,5.53251,5.89871,2.356936,Delta


In [9]:
os.chdir('/Users/macbookair/Downloads/GitHub/renewable/clusters_1km/')

In [10]:
clusters = pd.read_excel('clusters.xlsx')

In [12]:
clusters_1km = clusters.merge(solar_median, left_on ='adm1', right_on = 'adm1', how='left')
clusters_1km = clusters_1km.drop(['LAT','LON'],axis=1)

In [17]:
clusters_1km.to_excel('clusters.xlsx',index=False)